<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC', 'Inconsolata'; font-size: 600%; font-weight: 700; text-align: center; color: DarkSeaGreen;">
自訂set實作
</div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 實作完整的set談何容易，要顧及的底層細節非常多，很可能超出筆者的能力。
* 所以本文的自訂set，只是修改Python set的一小部分，練功成份大於實用。

</div>
<br><br>

In [ ]:
# from colors import Colors
# import random as rd

# print(f'{Colors.Fore.MAGENTA}{Colors.Back.YELLOW}*黃底洋紅字*')
# print(f'\033[38;2;229;198;105m**設定輸出顏色**')
# print(f'\033[38;2;255;215;0mGold Gold')
# print(f'{chr(27)}[38;2;{(r := rd.randint(0, 255))};{(g := rd.randint(0, 255))};{(b := rd.randint(0, 255))}m\x1b[48;2;{r};{g};{b}mHello, world.')


In [95]:
%%javascript
// 設定output文字顏色
document.styleSheets[0].addRule('body', 'color: #E594BD !important;')

<IPython.core.display.Javascript object>

In [86]:
print('Hello')

Hello


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
先複習Python的set
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set的notation
* set的特性
* 建立空set
* 運算子和方法


<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
set的notation
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set是用大括號(花括弧)`{}`作為前後綴符號，例如：
* `elements = {'Python', 'JS', 'C++', 'Java'}`

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
set的特性
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 每個元素都是唯一(unique)，不會重複。
* 元素的順序無法保證(unordered)。 
* 可新增、刪除元素，但不能修改元素的值。
* 可用迴圈(通常用for loop)迭代。所以set是`iterable`。
* 由於不保證元素位置，set並無索引(index)，當然也沒有切片(slice)。所以set非`sequence`。
* set的元素不可以是unhashable物件。

<div style="color: Turquoise; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 200%; font-weight: 700;">
補充說明
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* `iterable` vs `sequence`：
    * `iterable`在這裡是名詞，非形容詞。意指「可迭代的物件」。
    * `sequence`是`iterable`的一個「特例」(a more specific type of iterable)，特點是有索引，可以進行切片運算(slicing operations)。
    * 從set theory角度看這兩者，可以說`sequences`是`iterables`的subset，`iterables`則是`sequences`的superset。
* 我們說某物件是hashable，意思是它有一個`hash value`，這個value在該物件的生命周期內都不會改變。An object is hashable if it has a hash value that does not change during its lifetime.
* 一個簡易但不精準的識別hashability方法是：immutable物件大致上是hashable，mutable物件則多半是unhashable。兩者高度相關，但非絕對。誠如ChapGPT所說：
    <div style="color: LightSkyBlue; font-size: 90%;">
        
    > ...while there is a strong correlation between immutability and hashability in Python, the rule is not absolute. Hashability is more about whether an object’s hash value remains constant throughout its lifetime rather than just its mutability or immutability status.
    </div>

In [ ]:
from typeutil import is_hashable

t1 = (1, 2, 3)    # tuple is immutable
is_hashable(t1)

t2 = (1, 2, [])   # tuple is immutable
is_hashable(t2)

is_hashable([])

In [ ]:
t2 = (1, 2, [])
t3 = (t2, 1)
is_hashable(t3)

In [ ]:
# 每個元素都是唯一(unique)，不會重複。
elements = {1, 9, 6, 4, 1, 4, 1}
print(f'{elements = }')

In [ ]:
# 元素的順序無法保證(unordered)。
elements = {3, 3.1416, '0', None, True, ()}
print(f'{elements = }')

In [ ]:
# 可新增、刪除元素，但不能修改元素的值。
...   # 稍後詳說


In [ ]:
# 可用迴圈迭代。
elements = {8, 3, '0', None, True}
for element in elements:
    print(f'{element = }')

In [ ]:
# 沒有索引(index)和切片(slice)。
elements = {0, 1, 2, 3, 4}
print(f'{elements = }')
# print(elements[1])
print(elements[:1])

In [ ]:
# 元素不可以為unhashable物件。
# elements = {2, '4', True, None, ()}
elements = {2, '4', True, None, (), ((),)}
print(f'{elements = }')

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
建立空set
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 建立空set請用`set1 = set()`。
* 不可以用`set1 = {}`，因為這個寫法是建立空dict而非空set。
    * 建立空list可用`list1 = list()`或`list1 = []`。
    * 建立空tuple可用`tuple1 = tuple()`或`tuple1 = ()`。
    * 建立空dict可用`dict1 = dict()`或`dict1 = {}`。
    * 唯獨空set只能用`set()`來建。

<div style="text-align:center"><img src="https://hackmd.io/_uploads/SyI5hFsS3.png" width="400"/></div>

In [ ]:
# 建立空set。
set1 = set()
set2 = {}   # empty dict, NOT empty set

print(f'{set1 = }\t{type(set1) = }')
print(f'{set2 = }\t{type(set2) = }')

<div style="text-align:center"><img src="https://hackmd.io/_uploads/ryRk_EdwT.jpg" width="400"/></div>

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
運算子和方法
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set提供以下運算子(operators)和對應方法(methods)：
    <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 85%; color: Gainsboro">

    |運算子|方法|中文|範例 / 說明|
    |--|--|--|--|
    |\||union()|聯集|<ol><li>set1 \| set2</li><li>set1.union(set2)</li></ol>|
    |&|intersection()|交集|<ol><li>set1 & set2</li><li>set1.intersection(set2)|
    |-|difference()|差集|<ol><li>set1 - set2</li><li>set1.difference(set2)|
    |^|symmetric_difference()|對稱差|<ol><li>set1 ^ set2</li><li>set1.symmetric_difference(set2)|
    |<=|issubset()|子集|<ol><li>set1 <= set2</li><li>set1.issubset(set2)|
    |<|proper subset<br>(no direct method)|真子集|<ol><li>set1 < set2</li><li>set1.issubset(set2) and<br>set1 != set2|
    |>=|issuperset()|超集|<ol><li>set1 >= set2</li><li>set1.issuperset(set2)|
    |>|proper superset<br>(no direct method)|真超集|<ol><li>set1 > set2</li><li>set1.issuperset(set2) and<br>set1 != set2|
    |\|=|update()||add elements from all  others.|
    |&=|intersection_update()||keep only elements found<br>in it and all others.|
    |-=|difference_update()||remove elements found in<br>others.|
    |^=|symmetric_difference_update()||keep only elements found in<br>either set, but not in both.|
    </div>
* 上表最後4項運算子`|=`, `&=`, `-=`, `^=`構成的是statement(指令/敘述)，其餘各項則構成expression(運算式)。
* Python的set沒有提供`+`、`*`或`/`運算子。
</div>

In [ ]:
# union
set1 = {1, 2, 3, 4, 5}
set2 = {8, 7, 6, 5, 4}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 | set2      = }')
print(f'{set1.union(set2) = }')
print(f'{set1 = }')
print(f'{set2 = }')

In [ ]:
# intersection
set1 = {1, 2, 3, 4, 5}
set2 = {8, 7, 6, 5, 4}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 & set2             = }')
print(f'{set1.intersection(set2) = }')
print(f'{set1 = }')
print(f'{set2 = }')

In [ ]:
# difference
set1 = {1, 2, 3, 4, 5}
set2 = {8, 7, 6, 5, 4}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 - set2           = }')
print(f'{set1.difference(set2) = }')
print(f'{set1 = }')
print(f'{set2 = }')

In [ ]:
# symmetric difference
set1 = {1, 2, 3, 4, 5}
set2 = {8, 7, 6, 5, 4}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 ^ set2                     = }')
print(f'{set1.symmetric_difference(set2) = }')
print(f'{set1 = }')
print(f'{set2 = }')

In [ ]:
# subset
set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 <= set2        = }')
print(f'{set1.issubset(set2) = }\n')

set1 = {3, 5, 1}
set2 = {1, 2, 3, 4, 5}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 <= set2        = }')
print(f'{set1.issubset(set2) = }\n')

set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3, 2, 1}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 <= set2        = }')
print(f'{set1.issubset(set2) = }')

In [ ]:
# proper subset
set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 < set2                            = }')
print(f'{(set1.issubset(set2) and set1 != set2) = }\n')

set1 = {3, 5, 1}
set2 = {1, 2, 3, 4, 5}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 < set2                            = }')
print(f'{(set1.issubset(set2) and set1 != set2) = }\n')

set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3, 2, 1}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 < set2                            = }')
print(f'{(set1.issubset(set2) and set1 != set2) = }')

In [ ]:
# superset
set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 >= set2          = }')
print(f'{set1.issuperset(set2) = }\n')

set1 = {3, 5, 1}
set2 = {1, 2, 3, 4, 5}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 >= set2          = }')
print(f'{set1.issuperset(set2) = }\n')

set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3, 2, 1}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 >= set2          = }')
print(f'{set1.issuperset(set2) = }')

In [ ]:
# proper superset
set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3}

print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 > set2                              = }')
print(f'{(set1.issuperset(set2) and set1 != set2) = }\n')

set1 = {3, 5, 1}
set2 = {1, 2, 3, 4, 5}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 > set2                              = }')
print(f'{(set1.issuperset(set2) and set1 != set2) = }\n')

set1 = {1, 2, 3, 4, 5}
set2 = {5, 4, 3, 2, 1}
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1 > set2                              = }')
print(f'{(set1.issuperset(set2) and set1 != set2) = }')

In [ ]:
# update(add elements from all others.)
set1 = {1, 2, 3, 4, 5}
set2 = {7, 6, 5, 4, 3}

print(f'{set1 = }  {(old_id1 := id(set1)) = }')
print(f'{set2 = }  {(old_id2 := id(set2)) = }\n')
# set1 |= set2
set1.update(set2)
print(f'{set1 = }  {(new_id1 := id(set1)) = }')
print(f'{set2 = }  {(new_id2 := id(set2)) = }')
print(f'{(old_id1 == new_id1) = }')    # True or False?
print(f'{(old_id2 == new_id2) = }')    # True

In [ ]:
# intersection_update(keep only elements found in it and all others.)
set1 = {1, 2, 3, 4, 5}
set2 = {7, 6, 5, 4, 3}

print(f'{set1 = }  {(old_id1 := id(set1)) = }')
print(f'{set2 = }  {(old_id2 := id(set2)) = }\n')
set1 &= set2
print(f'{set1 = }  {(new_id1 := id(set1)) = }')
print(f'{set2 = }  {(new_id2 := id(set2)) = }')
print(f'{(old_id1 == new_id1) = }')
print(f'{(old_id2 == new_id2) = }')

In [ ]:
# difference_update(remove elements found in others.)
set1 = {1, 2, 3, 4, 5}
set2 = {7, 6, 5, 4, 3}

print(f'{set1 = }  {(old_id1 := id(set1)) = }')
print(f'{set2 = }  {(old_id2 := id(set2)) = }\n')
set1 -= set2
# set1.difference_update(set2)
print(f'{set1 = }  {(new_id1 := id(set1)) = }')
print(f'{set2 = }  {(new_id2 := id(set2)) = }')
print(f'{(old_id1 == new_id1) = }')
print(f'{(old_id2 == new_id2) = }')

In [ ]:
# symmetric_difference_update(keep only elements found in either set,
# but not in both.)
set1 = {1, 2, 3, 4, 5}
set2 = {7, 6, 5, 4, 3}

print(f'{set1 = }  {(old_id1 := id(set1)) = }')
print(f'{set2 = }  {(old_id2 := id(set2)) = }\n')
set1 ^= set2
print(f'{set1 = }  {(new_id1 := id(set1)) = }')
print(f'{set2 = }  {(new_id2 := id(set2)) = }')
print(f'{(old_id1 == new_id1) = }')
print(f'{(old_id2 == new_id2) = }')

<div style="text-align:center"><img src="https://hackmd.io/_uploads/ryRk_EdwT.jpg" width="400"/></div>

<div style="color: DarkSalmon; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 250%; font-weight: 700;">
set的其他方法
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 除了以上的運算外，set還有好幾個方法可用。這些方法沒有對應的運算子：
    <div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 90%; color: Gainsboro">
    
    |方法|說明|備註|
    |--|--|--|
    |add(elem)|新增元素||
    |remove(elem)|刪除元素|Raises KeyError if<br>elem is not contained<br>in the set.|
    |discard(elem)|刪除元素|Remove element elem<br>from the set if it<br>is present.|
    |pop()|Remove and return an<br>arbitrary element<br>from the set.|Raises KeyError if<br>the set is empty.|
    |clear()|Remove all elements<br>from the set.||
    |isdisjoint(other)|Return True if the<br>set has no elements in<br>common with other. |Sets are disjoint if<br>and only if their<br>intersection is the<br>empty set.|
    |copy()|Return a shallow copy<br>of the set.||
    </div>
* 一開始講set特性時有提到不能修改元素的值，所以沒有replace()方法。

</div>

In [ ]:
# add()
my_set = {8, 2, 6, 9, 3}
print(f'Original set             : {my_set = }')
my_set.add(7)
print(f'After adding 7 to the set: {my_set = }')
my_set.add(5)
print(f'After adding 5 to the set: {my_set = }')
my_set.add([])

In [89]:
# remove()
try:
    my_set = {4, 6, 2, 8, 7}
    print(f'Original set   : {my_set = }')
    my_set.remove(8)
    print(f'After remove(8): {my_set = }')
    my_set.remove(0)
    print(f'After remove(0): {my_set = }')
except KeyError:
    print('Cannot remove(0) from the set.')

Original set   : my_set = {2, 4, 6, 7, 8}
After remove(8): my_set = {2, 4, 6, 7}
Cannot remove(0) from the set.


In [ ]:
# discard()
try:
    my_set = {4, 6, 2, 8, 7}
    print(f'Original set    : {my_set = }')
    my_set.discard(8)
    print(f'After discard(8): {my_set = }')
    my_set.discard(0)
    print(f'After discard(0): {my_set = }')
except KeyError:
    print('Cannot discard(0) from the set.')


In [ ]:
# pop(): Remove and return an arbitrary element from the set.
# Raises KeyError if the set is empty.
try:
    my_set = {2, 5, 0, 6, 1}
    print(f'Original set   : {my_set = }')
    removed_element = my_set.pop()
    print(f'Removed element: {removed_element = }')
    print(f'After pop : {my_set = }\n')
    removed_element = my_set.pop()
    print(f'Removed element: {removed_element = }')
    print(f'After pop : {my_set = }\n')
    empty_set = set()
    # empty_set = {}  # empty dict, NOT empty set
    print(f'Original set   : {empty_set = }')
    empty_set.pop()
    print('hello, world.')
except KeyError:
    print('Cannot pop from an empty set.')

In [ ]:
# clear(): Remove all elements from the set.
my_set = {2, 5, 0, 6, 1}
print(f'Original set      : {my_set = }')
print(f'{my_set.clear() = }')
print(f'After clear       : {my_set = }')
print(f'{my_set.clear() = }')
print(f'Clear an empty set: {my_set = }')

In [ ]:
# isdisjoint(): checks if two sets have no elements in common.
set1 = {1, 2, 3, 4, 5}
set2 = {6, 7, 8}
# set1.isdisjoint(set2)  # Using method
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1.isdisjoint(set2) = }\n')

set1 = {1, 2, 3, 4, 5}
set2 = {6, 7, 8, 2}
# set1.isdisjoint(set2)  # Using method
print(f'{set1 = }')
print(f'{set2 = }')
print(f'{set1.isdisjoint(set2) = }\n')

In [ ]:
# copy()
old_set = {1, 2, 3}
print('Before copy:')
print(f'  {old_set = }\t{id(old_set) = }')
new_set = old_set.copy()
print('After copy:')
print(f'  {old_set = }\t{id(old_set) = }')
print(f'  {new_set = }\t{id(new_set) = }')

<div style="text-align:center"><img src="https://hackmd.io/_uploads/S1ZVMQOP6.png" width="500"/></div>

<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
開始實作一個自訂的set
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* set的內建方法，有些得注意：
    * add()方法新增的元素如果是unhashable，會產生exception。
    * remove()找不到要remove的元素時會產生exception。
    * 空set要pop()也會產生exception。
    * 沒有replace()。
* 以下筆者試圖創建一個自訂的set類別，看能否處理以上的小問題，順便加強一下其他功能。    
* 再強調一次：
    * 實作完整的set談何容易，要顧及的底層細節非常多，很可能超出筆者的能力。
    * 所以本文的自訂set，只是修改Python set的一小部分，練功成份大於實用。


<div style="color: SteelBlue; font-family: 'Ubuntu Mono', 'Inconsolata', 'Noto Sans TC'; font-size: 300%; font-weight: 700;">
自訂set規格
</div>
<br>
<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%; color: Gainsboro">

* 名稱：`RobustSet`
* 加強功能：
    * add()方法新增的元素如果是unhashable，不會產生exception而是略過。而且add()方法允許一次新增多個values。
    * remove()找不到要remove的元素時，會略過不會產生exception。而且允許一次remove多個values。傳回值改為bool，代表是否remove掉所有傳入的元素。
    * discard()的傳回值也改為bool。
    * 空set要pop()不會產生exception。

* 新增功能：
    * replace()方法。
    * show()方法。
* 未完部分：
    * 除以上所述外，set原本的其他功能都來不及補齊。
    * 所以目前`RobustSet`只是個剛開始發展的雛型。


In [ ]:
from typeutil import is_hashable

class RobustSet(set):
    def __init__(self, s: set):
        super().__init__()
        self.__set = s

    def add(self, elements: list | tuple | set):
        '''override父類別的add方法'''
        tmp_set = self.__set.copy()
        # added_count = 0
        for element in (hashable_elements := [element for element in elements if is_hashable(element)]):
            self.__set.add(element)

        return len(self.__set) - len(tmp_set)

    def discard(self, element: set):
        '''override父類別的discard方法'''
        if element in self.__set:
            return_value = True
        else:
            return_value = False
        self.__set.discard(element)
        return return_value

    def remove(self, elements: list | tuple | set):
        '''override父類別的remove方法'''
        if set(hashable_elements := [element for element in elements if is_hashable(element)]).issubset(self.__set):
            return_value = True
        else:
            return_value = False

        for hashable_element in hashable_elements:
            if hashable_element in self.__set:
                self.__set.remove(hashable_element)
        return return_value

    def pop(self):
        '''override父類別的pop方法'''
        if self.__set:
            return_value = self.__set.pop()
        else:
            return_value = None
        return return_value

    def replace(self, old: list | tuple | set, new: object) -> None:
        '''新增replace方法'''
        self.remove(old)
        self.add(new)
        return None

    def show(self):
        '''新增show方法'''
        return self.__set


In [ ]:
# 建立物件並呼叫物件的show方法
my_set = RobustSet({4, 7, 3, 3, 2, 0, 6})
my_set.show()
my_set = RobustSet({1, 8, 3, 0})
my_set.show()

In [ ]:
# 呼叫overrided的add方法
my_set = RobustSet({1, 8, 7, 0, 9, 3})
my_set.show()
my_set.add([4, [8], -0, {1}, '9', 4, 3, 2, 3])
my_set.show()

my_set = {1, 8, 7, 0, 9, 3}
# 一般的set執行下面這列會產生exception。
my_set.add([8])

In [ ]:
# 呼叫overrided的discard方法
my_set.show()
print(f'{my_set.discard(45) = }')
my_set.show()

In [ ]:
# 呼叫overrided的remove方法
# 本cell示範remove方法的參數可以為list, tuple, or set。
my_set = RobustSet({1, 8, 7, 0, 9, 3})
my_set.show()
print(f'{my_set.remove([4, 8, []]) = }')
my_set.show()

my_set = {1, 8, 7, 0, 9, 3}
# 一般的set執行下面這列會產生exception。
my_set.remove([4, 8, []])


# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.show()
# my_set.remove((4, 8, []))
# my_set.show()

# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.show()
# my_set.remove({4, 8, []})
# my_set.show()


In [ ]:
# 呼叫overrided的remove方法
# 本cell示範存入參數全都刪除時，remove方法傳回True。
my_set = RobustSet({1, 8, 7, 0, 9, 3})
my_set.show()
print(f'{my_set.remove([3, 8]) = }')
my_set.show()

# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.show()
# my_set.remove((3, 8))
# my_set.show()

# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.show()
# my_set.remove({3, 8})
# my_set.show()

In [ ]:
# 呼叫overrided的pop方法
# my_set = RobustSet({1, 8, 7, 0, 9, 3})
# my_set.show()
# my_set.pop()
# my_set.show()

my_set = RobustSet({})
my_set.show()
# pop空set不會產生exception。
my_set.pop()
my_set.show()

my_set = set()
# 一般的set執行下面這列會產生exception。
my_set.pop()

In [94]:
# 呼叫新增的replace方法。
my_set = RobustSet({1, 8, 7, 0, 9, 3})
my_set.show()
my_set.replace([0], ('Alex',))
my_set.show()

{0, 1, 3, 7, 8, 9}

{1, 3, 7, 8, 9, 'Alex'}